<a href="https://colab.research.google.com/github/shayleaschreurs/mod_13/blob/main/Colab_multiclass_in_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass Prediction in Deep Learning Models

## Read in the Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving meet_or_beat.csv to meet_or_beat.csv


In [ ]:
df = pd.read_csv('meet_or_beat.csv')

In [ ]:
df.shape

(71968, 5)

In [ ]:
df.head(3)

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,2.01,1.67,11.0,0.051444,0.018585
1,0.17,0.19,6.0,0.112955,-0.000510
2,-0.07,0.14,4.0,0.077167,-0.046104


## Preparing the Data

In [ ]:
# Generate the categorical outcome variable
df['earnings_outcome'] = np.nan
df.loc[(df['EPS']==df['forecasted_eps']), 'earnings_outcome'] = 'meet'
df.loc[(df['EPS']>df['forecasted_eps']), 'earnings_outcome'] = 'beat'
df.loc[(df['EPS']<df['forecasted_eps']), 'earnings_outcome'] = 'lose'
df.head()

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns,earnings_outcome
0,2.01,1.67,11.0,0.051444,0.018585,beat
1,0.17,0.19,6.0,0.112955,-0.000510,lose
2,-0.07,0.14,4.0,0.077167,-0.046104,lose
3,0.48,0.51,8.0,-0.006130,-0.004899,lose
4,-0.24,-0.27,9.0,0.089762,-0.025466,beat


In [ ]:
# Preview the output variable 
y = df['earnings_outcome']
y

0        beat
1        lose
2        lose
3        lose
4        beat
         ... 
71963    beat
71964    beat
71965    lose
71966    beat
71967    meet
Name: earnings_outcome, Length: 71968, dtype: object

In [ ]:
# Encode earnings labels to integers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([0, 1, 1, ..., 1, 0, 2])

In [ ]:
# Save the unique number of labels for future use
number_of_classes = len(list(y.drop_duplicates()))
number_of_classes

3

In [ ]:
# Convert labeled integers to a Keras `categorical` data type
from keras.utils.np_utils import to_categorical
y_categorical = to_categorical(encoded_y, num_classes=number_of_classes)


In [ ]:
# Specify X (predictor) variables
X = df[['forecasted_eps',
              'before_total_returns','noOfEsts']]
X.head(3)

,forecasted_eps,before_total_returns,noOfEsts
0,1.67,0.018585,11.0
1,0.19,-0.000510,6.0
2,0.14,-0.046104,4.0


In [ ]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=1)

In [ ]:
# Preview the encoded data we're trying to predict
pd.DataFrame(y_train).head(5)

,0,1,2
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0


In [ ]:
# Check for class balance
pd.DataFrame(y_train).sum()

0    32924.0
1    16878.0
2     4174.0
dtype: float32

In [ ]:
# Save the count of unique predictor variables for use in model
number_of_predictors = len(X.columns)
len(X.columns)

3

## Building, Fitting and Predicting with Multiple Classes

In [ ]:
# Build the neural network layers
model = Sequential()
model.add(Dense(9, input_dim=number_of_predictors, activation='relu'))
model.add(Dense(6, activation='relu'))

In [ ]:
# Add the final output layer
# With the # of possible outputs equal to the number classes 
model.add(Dense(number_of_classes, activation='softmax'))

In [ ]:
# Compile the model (with multi-class specific parameters)
model.compile(loss="categorical_crossentropy", 
              optimizer= "adam", 
              metrics=['categorical_accuracy'])


In [ ]:
# Summarise the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 36        
                                                                 
 dense_1 (Dense)             (None, 6)                 60        
                                                                 
 dense_2 (Dense)             (None, 3)                 21        
                                                                 
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fit the model
model.fit(X_train,y_train, 
                    epochs=20,
                    batch_size=1000,
                    shuffle=True)

Epoch 1/20
54/54 [==============================] - 1s 2ms/step - loss: 2.0748 - categorical_accuracy: 0.1014
Epoch 2/20
54/54 [==============================] - 0s 2ms/step - loss: 1.3591 - categorical_accuracy: 0.2988
Epoch 3/20
54/54 [==============================] - 0s 2ms/step - loss: 1.0090 - categorical_accuracy: 0.4668
Epoch 4/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8914 - categorical_accuracy: 0.6095
Epoch 5/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8669 - categorical_accuracy: 0.6099
Epoch 6/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8609 - categorical_accuracy: 0.6100
Epoch 7/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8571 - categorical_accuracy: 0.6100
Epoch 8/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8525 - categorical_accuracy: 0.6100
Epoch 9/20
54/54 [==============================] - 0s 2ms/step - loss: 0.8484 - categorical_accuracy: 0.6099
Epoch 10/2

In [ ]:
# Evaluate model on the test data
model.evaluate(X_test,y_test, verbose=2)

563/563 - 1s - loss: 0.8395 - categorical_accuracy: 0.6155 - 777ms/epoch - 1ms/step


[0.8395379185676575, 0.6154958009719849]

In [ ]:
# Save predictions on the test data
predictions = model.predict(X_test)
predictions

563/563 [==============================] - 1s 1ms/step


array([[0.5742632 , 0.32977974, 0.09595694],
       [0.7637234 , 0.1644749 , 0.07180177],
       [0.5826478 , 0.35416842, 0.06318378],
       ...,
       [0.64515114, 0.28018785, 0.07466098],
       [0.56804997, 0.34906146, 0.0828886 ],
       [0.5159607 , 0.40440705, 0.07963229]], dtype=float32)

In [ ]:
# Get the most likely prediction for each observation
most_likely = np.argmax(predictions, axis=1)
most_likely

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# Convert most likely category back to original labels
most_likely = encoder.inverse_transform((most_likely))
most_likely

array(['beat', 'beat', 'beat', ..., 'beat', 'beat', 'beat'], dtype=object)

In [ ]:
# Evaluate prediction balance
pd.DataFrame(most_likely).value_counts()

beat    17835
lose      157
dtype: int64